# Processing cellpy batch - cycles

### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`

**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`   
**Date:** `{{cookiecutter.date}}`  

## Imports and setup

In [ ]:
import os
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
from holoviews import opts

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch, helpers, plotutils

In [ ]:
%matplotlib inline
hv.extension('bokeh')
pd.set_option('display.max_columns', 70)
print(f"cellpy version: {cellpy.__version__}")

In [ ]:
cellpy_batch_file = "cellpy_batch_" + "{{cookiecutter.session_id}}" + ".json"

In [ ]:
b = batch.from_journal(cellpy_batch_file)
b.link()
cell_labels = b.experiment.cell_names
print(" cell labels: ")
cell_labels

In [ ]:
# # Remove bad cycles (if you have saved it to the journal session)
# helpers.yank_outliers(b, remove_indexes=b.journal.session["bad_cycles"])

In [ ]:
# # It is usually nice to see the capacity plot in the same notebook.
# # You can use the b.plot_summary() method or plotutils.bplot e.g.
# plotutils.bplot(b, columns=["charge_capacity"], max_cycle=100)

## Plot all cycles

This will output an interactive plot for all the cycles for each cell. If you want to only plot selected cycles you can use the `cycle` argument in the `get_cap` method, *e.g.*
```python
curves = d.get_cap(cycle=[1, 2, 3, 10, 100])
```

In [ ]:
# plot all cycles up to cycle 50
voltage_curves = dict()
cycle = list(range(1, 51))
for label in cell_labels:
    d = b.experiment.data[label]
    curves = d.get_cap(label_cycle_number=True, interpolated=True, number_of_points=100)
    if not curves.empty:
        curve = hv.Curve(curves, kdims=["capacity", "cycle"], vdims="voltage").groupby("cycle").overlay()#.opts(show_legend=False)
        voltage_curves[label] = curve
    else:
        print(f"[{label} empty]")
NdLayout_voltage_curves = hv.NdLayout(voltage_curves, kdims='label').cols(3)

In [ ]:
palette = 'Spectral'
NdLayout_voltage_curves.opts(
    hv.opts.Curve(color=hv.Palette(palette), tools=['hover']),
    hv.opts.NdOverlay(shared_axes=False),
    hv.opts.NdLayout(),
)

## Making and saving cycles to html and pickle for later use

In [ ]:
cell_id = b.experiment.cell_names[0]
c = b.experiment.data[cell_id]

In [ ]:
cycle = [1,2, 10, 20]

curves = d.get_cap(label_cycle_number=True, interpolated=True, number_of_points=100)
curves_fig = hv.Curve(curves, kdims=["capacity", "cycle"], vdims="voltage").groupby("cycle").overlay()#.opts(show_legend=False)
curves_fig

In [ ]:
fig_label = f"{{cookiecutter.session_id}}_{cell_id}_curves_evolution"

# export to html:
hv.save(
    curves_fig,
    f"out/{fig_label}.html",
    toolbar=True
)

# save as pickle (can be loaded in other notebooks)
Pickler.save(curves_fig, f"out/{fig_label}.hvz",)

## Select the main curves and save them

For example:  
- make a list with only the cell labels you want
- make a list with the cycles you want
- iterate and combine them into either one big dataframe or a list of list

If you make a list of lists, you can save it using the `itertools.zip_longest` and `csv` method as described in the notes notebook ([link](00_{{cookiecutter.notebook_name}}_notes.ipynb)).

## Links

### Notebooks
- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)